# Import the packages

In [2]:
# Importing necessary libraries

# Core Python packages and utilities
import os
import gc
import warnings
import numpy as np
from scipy.stats import norm

# Data handling and preprocessing
import pandas as pd
from pandas.errors import PerformanceWarning
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Time series analysis and modeling
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import het_arch
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from arch import arch_model
from prophet import Prophet

# Machine learning libraries
from sklearn.linear_model import LinearRegression
from scipy.optimize import nnls
from xgboost import DMatrix
import xgboost as xgb
import lightgbm as lgb

# Neural networks and deep learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
import tensorflow as tf
from optuna.integration import TFKerasPruningCallback
import optuna

# Visualization
import matplotlib.pyplot as plt

# Parallel computing
import pyopencl as cl

# Quality of life settings

In [3]:
# Suppress warnings (e.g., FutureWarnings, PerformanceWarnings)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=PerformanceWarning)
warnings.filterwarnings("ignore", message="No frequency information was provided, so inferred frequency 30min will be used.")

# Parallel computing configuration for TensorFlow
num_of_threads = 8  # Set the number of threads manuallly according to your PC specifications
tf.config.threading.set_intra_op_parallelism_threads(num_of_threads)

# Checks which devices are available and sets the cuda gpu in path
platforms = cl.get_platforms()
for i, platform in enumerate(platforms):
    print(f"Platform {i}: {platform.name}")
    for j, device in enumerate(platform.get_devices()):
        if platform.name == "NVIDIA CUDA":
            gpu_platform_id = i
            gpu_device_id = j
        print(f"  Device {j}: {device.name}")
os.environ["LIGHTGBM_GPU_PLATFORM_ID"] = str(gpu_platform_id)  # Makes sure the Platform id that will be used is set correctly in the System Varibles
os.environ["LIGHTGBM_GPU_DEVICE_ID"] = str(gpu_platform_id)  # Makes sure the GPU id that will be used is set correctly in the System Varibles
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Platform 0: NVIDIA CUDA
  Device 0: NVIDIA GeForce GTX 1060
Platform 1: Intel(R) OpenCL HD Graphics
  Device 0: Intel(R) UHD Graphics 620


# Import the data

In [68]:
merged_df = pd.read_csv("merged_df_Combined.csv", parse_dates=['Datetime'])  # df merging four initial csvs with missing filled from previous preprocessing steps
merged_df.set_index('Datetime', inplace=True)
merged_df_2024 = merged_df[(merged_df.index >= '2023-09-30') & (merged_df.index < '2025-01-01')]  # we only look at the last 365 days

# User Defined Functions for quality of life reasons

In [5]:
def calculate_fft(series, n_top_seasonalities, threshold_pc=0.02):
    """
    Calculate significant positive frequencies and their amplitudes using Fast Fourier Transform (FFT),
    selecting the lower of 2% of the max amplitude or the top `n` frequencies.

    Parameters:
    - series (pd.Series): The input time series data.
    - n_top_seasonalities (int): The maximum number of significant frequencies to consider.
    - threshold_pc (float): Percentage (0 < threshold_pc <= 1) of the maximum amplitude to filter significant frequencies.

    Returns:
    - zip: A generator yielding (positive frequency, amplitude) for each significant frequency.
    """
    # Compute fast Fourier transform
    price_fft = np.fft.fft(series.dropna())

    # Get frequencies corresponding to FFT coefficients
    freqs = np.fft.fftfreq(len(price_fft), d=1/48)  # because half-hourly data

    # Calculate amplitudes
    amplitudes = np.abs(price_fft)

    # Calculate the threshold based on 2% of the max amplitude
    threshold = threshold_pc * np.max(amplitudes)

    # Filter positive frequencies with amplitudes above threshold
    positive_indices = np.where((amplitudes > threshold) & (freqs > 0))
    positive_freqs = freqs[positive_indices]
    positive_amplitudes = amplitudes[positive_indices]

    # Sort by amplitude and select the lower of `n_top_seasonalities` or all significant frequencies
    sorted_indices = np.argsort(positive_amplitudes)[::-1]
    selected_indices = sorted_indices[:min(n_top_seasonalities, len(sorted_indices))]

    # Select the top frequencies and amplitudes
    significant_freqs = positive_freqs[selected_indices]
    significant_amplitudes = positive_amplitudes[selected_indices]

    return zip(significant_freqs, significant_amplitudes)


def metrics(y_test, y_pred):
    """
    Calculate and display error metrics for model evaluation.

    This function computes standard error metrics, including Mean Absolute Error (MAE), Mean Squared Error (MSE),
    Root Mean Squared Error (RMSE), and R-squared (R2), to evaluate the accuracy of model predictions.

    Parameters:
    - y_test (Series or array-like): The true values for the target variable in the test set.
    - y_pred (Series or array-like): The predicted values for the target variable in the test set.

    Returns:
    - None
    """
    # Calculate error metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Print error metrics
    print("Mean Absolute Error (MAE):", mae)
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R-squared (R2):", r2)
    return rmse


# Memory reduction for our dataset by setting optimal dtypes
def reduce_mem_usage(df, verbose=True):
    """
    Reduces the memory usage of a pandas DataFrame by downcasting numeric columns 
    to the smallest possible dtype that can fit the range of values, and converting 
    other columns to 'category' type where applicable.

    Parameters:
    df (pd.DataFrame): The input DataFrame to optimize.
    verbose (bool): If True, prints the memory usage reduction.

    Returns:
    pd.DataFrame: The optimized DataFrame with reduced memory usage.

    Function Details:
    - Identifies numeric columns with data types like integers and floats.
    - For integer columns, downcasts to the smallest possible integer type 
      (e.g., int8, int16, int32, int64) that can accommodate the data's range.
    - For float columns, downcasts to the smallest possible float type 
      (e.g., float16, float32, float64) that can accommodate the data's range.
    - Converts non-numeric columns to 'category' type for memory efficiency.
    - Calculates and optionally prints the reduction in memory usage.

    Notes:
    - Ensures no data loss during type conversion by checking the range of values 
      before downcasting.
    - Particularly useful for large datasets where memory constraints are a concern.
    """
    numerics = ["int8", 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
             df[col] = (df[col]).astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


# Plotting series, ACF, and PACF
def ACF_PACF(residuals):
    """
    Plots the Time Series, Auto-Correlation Function (ACF), and Partial Auto-Correlation Function (PACF)
    for a given set of residuals to analyze time series behavior.

    Parameters:
    residuals (pd.Series): A pandas Series containing the residuals or time series data.

    Returns:
    None: The function displays the plots but does not return any values.

    Function Details:
    - Creates a 3-row subplot layout for visual analysis.
    - The first plot shows the time series of the residuals to observe any patterns or trends over time.
    - The second plot displays the ACF (Auto-Correlation Function) to measure the correlation between 
      the time series and its lagged values.
    - The third plot displays the PACF (Partial Auto-Correlation Function) to measure the correlation 
      between the time series and its lagged values while controlling for the correlations of intervening lags.
    - The number of lags to display in ACF and PACF can be adjusted in the `lags` parameter.
    - Uses `tight_layout()` to optimize spacing between subplots for better visualization.
    """
    # Create the figure and axes
    fig, axes = plt.subplots(3, 1, figsize=(12, 16))

    # Plot the time series
    axes[0].plot(residuals.index, residuals, color='blue')
    axes[0].set_title('Time Series')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Value')

    # Plot ACF (Auto-Correlation Function)
    plot_acf(residuals, ax=axes[1], lags=100)  # Set the lags as per your requirement
    axes[1].set_title('ACF (Auto-Correlation Function)')

    # Plot PACF (Partial Auto-Correlation Function)
    plot_pacf(residuals, ax=axes[2], lags=100)  # Set the lags as per your requirement
    axes[2].set_title('PACF (Partial Auto-Correlation Function)')

    # Adjust layout for better appearance
    plt.tight_layout()
    plt.show()
    

def train_test_split_index_univariate(data, test_size=48):
    """
    Splits a dataset into sequential training and testing sets.

    Parameters:
    data (pd.DataFrame or pd.Series): Dataset to split.
    test_size (int): Number of samples in the test set (default: 48).

    Returns:
    tuple: (train, test) where:
        - train: All rows except the last `test_size` rows.
        - test: Last `test_size` rows.

    Notes:
    - Preserves order, suitable for time-series data.
    """
    train = data.iloc[:-test_size]
    test = data.iloc[-test_size:]
    return train, test


def train_test_split_index_multivariate(data, test_size=48):
    """
    Splits a dataset into sequential training and testing sets.

    Parameters:
    data (pd.DataFrame or pd.Series): Dataset to split.
    test_size (int): Number of samples in the test set (default: 48).

    Returns:
    tuple: (train, test) where:
        - train: All rows except the last `test_size` rows.
        - test: Last `test_size` rows.

    Notes:
    - Preserves order, suitable for time-series data.
    """
    data = data.iloc[2:-(test_size+1)]  # remove the missing because of lags and the predicted
    train = data.iloc[:-test_size]
    test = data.iloc[-test_size:]
    return train, test


from sklearn.preprocessing import PowerTransformer



def Yeo_Johnson(series):
    pt = PowerTransformer(method='yeo-johnson')
    transformed = pt.fit_transform(series.values.reshape(-1, 1)).flatten()
    return pd.Series(transformed, index=pd.to_datetime(series.index)), pt

def inverse_Yeo_Johnson(transformed_series, pt):
    # Reshape the transformed data to 2D
    transformed_2d = transformed_series.values.reshape(-1, 1)
    
    # Apply the inverse transformation
    original_data = pt.inverse_transform(transformed_2d)
    
    # Flatten the result and return as a Pandas Series
    return pd.Series(original_data.flatten(), index=transformed_series.index)


def plot_results(test, test_original, test_predictions, model_name):
    plt.figure(figsize=(20, 10))
    plt.plot(test.index, test_original, label='Actual Data', color='blue')
    plt.plot(test.index, test_predictions, label=f'{model_name} Predictions', linestyle='dashed', color='red')
    plt.title(f'{model_name} Predictions vs Actual Data')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# User Defined Functions for Feature Engineering

In [ ]:
from statsforecast.mfles import MFLES # Import MFLES directly
def univariate_MFLES(series, num_rows):
    temp_index = series.index
    series, pt = Yeo_Johnson(series)
    series.index.freq = pd.infer_freq(temp_index)
    # Initialize and fit MFLES
    mfles_model = MFLES(verbose=1)

    # Define seasonal periods and multiple `ma` values
    seasonal_periods = [48, 48*7, 48*30, 48*366, 12, 16]
    ma_values = [3, 4, int(min(seasonal_periods)), int(min(seasonal_periods)/2),None]

    # Perform optimization
    optimal_params = mfles_model.optimize(
        y=series.values,  # Time series as a NumPy array
        test_size=48*10,  # Test set size
        n_steps=3,  # Number of train-test splits
        seasonal_period=seasonal_periods,  # Seasonal periods
        metric="smape",  # Metric for evaluation
        params={
            "ma": ma_values,  # Include multiple `ma` values
            "smoother": [True],
            "seasonality_weights": [True],
            "changepoints": [False],
        },
    )

    # Fit the model using optimal parameters
    fitted_values = mfles_model.fit(
        y=series.values,
        seasonal_period=optimal_params.get("seasonal_period", seasonal_periods),
        max_rounds=10,  # Adjust this if needed
        ma=optimal_params.get("ma"),
        smoother=optimal_params.get("smoother"),
        seasonality_weights=optimal_params.get("seasonality_weights"),
        changepoints=optimal_params.get("changepoints"),
    )


    # Predict out-of-sample (next 48 steps)
    forecast_horizon = num_rows
    future_predictions = mfles_model.predict(forecast_horizon)


    # Combine original series and predictions for plotting
    future_index = pd.date_range(start=temp_index[-1], periods=forecast_horizon+1, freq='30T')[1:]
    future_series = pd.Series(future_predictions, index=future_index)

    # Align fitted with series index
    fitted_series = pd.Series(fitted_values, index=temp_index)

    # Combine fitted_values and predicted
    combined_series = pd.concat([fitted_series, future_series])


    #########################################################################
    # Step 2: Calculate Residuals
    residuals = series - fitted_values

    # Step 3: Fit ARIMA on the Residuals
    arima_forecast = statsforecast_arima(residuals, num_rows)  # Ensure non-na data for ARIMA

    # Step 4: Combine the Predictions of Prophet and ARIMA
    combined_forecast = combined_series.add(arima_forecast)  # Using fill_value to handle NaNs

    #########################################################################

    # Plot

    # plt.plot(series, label='Original Series')
    # plt.plot(combined_series, linestyle='dashed', color='red', label='Fitted + Predicted')
    # plt.legend()
    # plt.show()

    # Ensure fitted and series have the same length for comparison
    common_arima = combined_forecast[:len(series)]
    
    # Calculate MAPE without ARIMA
    actual_series = inverse_Yeo_Johnson(series, pt)
    fitted_series_inv = inverse_Yeo_Johnson(fitted_series, pt)
    # Calculate MAPE with ARIMA
    common_arima_inv = inverse_Yeo_Johnson(common_arima, pt)

    # Calculate SMAPE without ARIMA
    smape_without_arima = (
        np.mean(2 * np.abs(actual_series - fitted_series_inv) / (np.abs(actual_series) + np.abs(fitted_series_inv)))
    ) * 100
    print(f"SMAPE in-sample without ARIMA: {smape_without_arima:.2f}%")

    # Calculate SMAPE with ARIMA
    smape_with_arima = (
        np.mean(2 * np.abs(actual_series - common_arima_inv) / (np.abs(actual_series) + np.abs(common_arima_inv)))
    ) * 100
    print(f"SMAPE in-sample with ARIMA: {smape_with_arima:.2f}%")

    return inverse_Yeo_Johnson(combined_forecast, pt)


def extend_df_MFLES_univariate(df, num_rows):
    to_predict = ["Intraday_Price", "Total_Load","Demand_Outturn", "System_Price", "NIV_Outturn",
                  # "BSAD_Turn_Up","BSAD_Turn_Down",
                  "EPEX_Intraday_Volume",  "Wind_Solar", "Total_Generation",
                  # "Biomass", "Fossil_Gas", "Fossil_Hard_Coal", "Hydro_Pumped_Storage", "Hydro_Run-of-River_and_Poundage", "Nuclear", "Solar", "Wind_Onshore", "Wind_Offshore"  # Energy generation which is cyclical, except Fossil Oil which is WN
                  ]  # Also don't predict Loss of Load Prob which is WN
    # to_derive = ["BSAD_Total"]  # Calculate as the sum of BSAD_Turn_Up and BSAD_Turn_Down

    # Creating rows to append generated predictions
    # Generate a date range that starts after the last date in df, respecting the half hour datetime
    date_range_df_temp = pd.DataFrame({'value': [None] * num_rows}, index=pd.date_range(start=df.index[-1] + pd.Timedelta(minutes=30), periods=num_rows, freq='30T'))

    # Concatenate without resetting the index, preserving the datetime index
    df_with_preds = pd.concat([df, date_range_df_temp])
    date_range_df_temp = date_range_df_temp.drop(columns=["value"])  # Remove the temporary column we created in order to introduce rows

    for col in to_predict:
        print("Working on", col)
        df_with_preds.loc[df_with_preds.index[-num_rows:], col] = univariate_MFLES(df[[col]], num_rows)[-num_rows:].values

    # df_with_preds[to_derive[0]] = df_with_preds["BSAD_Turn_Up"] + df_with_preds["BSAD_Turn_Down"]  # calculate the BSAD toal wiht the two columns that make it up

    not_predicted_columns = set(df_with_preds.columns) - set(to_predict)  # Columns for which predictions are basically just their respective mean

    return df_with_preds, list(not_predicted_columns)


def feature_engineering(df):
    # Feature engineering to create wind+solar variable, ignoring NaNs (if there is NaN in one of them, the sum is not NaN)
    df.loc[:,"Wind_Solar"] = df[["Solar", "Wind_Onshore", "Wind_Offshore"]].sum(axis=1, skipna=True)
    # Sum all columns except 'GMT Time', ignoring NaNs
    df.loc[:,'Total_Generation'] = df.sum(axis=1, skipna=True)

    df_with_preds, not_predicted_columns = extend_df_MFLES_univariate(df, 49)  # takes in the df and appends new rows, fills with LSTM for select columns, 49 becaus of how we set up LSTM

    df_with_preds.drop(columns=["value"], inplace=True, errors="ignore")  # make sure the placeholder had been deleted

    # Variable that tracks the difference between total load and demand
    df_with_preds["Load-Demand"] = df_with_preds["Total_Load"] - df_with_preds["Demand_Outturn"]

    # Recalculate the LoLP using the Normal CDF (we only use the difference btw Total load and demand outturn, not entirely correct, but Loss of Load probability had a lot of missing values so we wanted to calculate something of our own). 
    # For instructions, we consulted:
    # approximation since we do not have some of the information
    # https://bscdocs.elexon.co.uk/category-3-documents/loss-of-load-probability-calculation-methodolgy-statement
    df_with_preds['LoLP'] = 1 - norm.cdf(df_with_preds["Load-Demand"], loc=0, scale=np.sqrt(700))
    # Calculate the LoLP lag 1 as proxy for prediction of not enough electricity for next day since the load and demand are super autocorellated
    df_with_preds['LoLP_lag1'] = df_with_preds['LoLP'].shift(1)


    # Feature engineering to create wind+solar variable, ignoring NaNs (if there is NaN in one of them, the sum is not NaN)
    df_with_preds["Wind_Solar"] = df_with_preds[["Solar", "Wind_Onshore", "Wind_Offshore"]].sum(axis=1, skipna=True)
    # Sum all columns except 'GMT Time', ignoring NaNs
    df_with_preds['Total_Generation'] = df_with_preds.sum(axis=1, skipna=True)


    # Total_Load = Total Generation + Exports - Imports - Stored Energy
    # So we create a column that is the difference between exports, imports and stored energy
    df_with_preds["Exports-Imports-Stored"] = df_with_preds["Total_Load"] - df_with_preds["Total_Generation"]
    df_with_preds["Generation-Demand"] = df_with_preds["Total_Generation"] - df_with_preds["Demand_Outturn"]
    
    Day_Ahead_Price_lag_48 = df_with_preds["Day_Ahead_Price"].shift(48)

    for col in df_with_preds:
        if df_with_preds[col].dtype == "object":
            if col != "value":
                df_with_preds[col] = pd.to_numeric(df_with_preds[col], errors='coerce')
    
    df_with_preds['Datetime'] = df_with_preds.index

    df_with_preds['day_of_week'] = df_with_preds['Datetime'].dt.dayofweek
    df_with_preds['is_weekend'] = (df_with_preds['Datetime'].dt.weekday >= 5).astype('int8')  # Monday (0) to Friday (4)

    df_with_preds['hour_of_day'] = df_with_preds['Datetime'].dt.hour
    df_with_preds['hour_of_day_half'] = df_with_preds['Datetime'].dt.hour + df_with_preds['Datetime'].dt.minute / 60

    df_with_preds['is_peak_hour'] = df_with_preds['hour_of_day'].isin([16, 17, 18, 19, 20]).astype('int8')
    df_with_preds['is_night'] = df_with_preds['hour_of_day'].isin([0, 1, 2, 3, 4, 5, 23]).astype('int8')

    # Create a boolean column 'close_price' for the last observation of the day
    df_with_preds['close_price'] = (df_with_preds['Datetime'].dt.time == pd.Timestamp('23:30').time()).astype('int8')
    # Create a new column for prev_day_close
    df_with_preds['prev_day_close'] = None
    # Assign the shifted close prices to the 'prev_day_close' column
    df_with_preds.loc[:, 'prev_day_close'] = df_with_preds.loc[df_with_preds['close_price'] == 1, "Intraday_Price"].shift()
    # Fill missing values in the 'prev_day_close' column with backward fill
    df_with_preds['prev_day_close'] = df_with_preds['prev_day_close'].fillna(method='bfill')
    # Create a boolean column 'open_price' for the first observation of the day
    df_with_preds['open_price'] = (df_with_preds['Datetime'].dt.time == pd.Timestamp('00:00').time()).astype('int8') * df_with_preds['prev_day_close']

    df_with_preds["System_Price_lag_1"] = df_with_preds["System_Price"].shift(1)
    df_with_preds["System_Price_lag_2"] = df_with_preds["System_Price"].shift(2)
    df_with_preds["NIV_Outturn_lag_1"] = df_with_preds["NIV_Outturn"].shift(1)
    df_with_preds["NIV_Outturn_lag_2"] = df_with_preds["NIV_Outturn"].shift(2)


    # seasonalities using the results from the fft
    freqs = []
    for freq, amp in calculate_fft(df_with_preds["System_Price"], 3, threshold_pc=0.02):  # Outputs same seasonalities for System Price and NIV_outturn
        freqs.append(freq)
    freqs = np.array(freqs)
    all_periods_in_days = 1 / freqs
    # Multiply each unique period by 24 to get values in hours
    unique_cycles_in_days = np.unique(all_periods_in_days.round(2))
    unique_cycles_in_hours = unique_cycles_in_days * 48  # because half hourly data

    # Format the results from scientific notation to decimal for readability
    formatted_cycles = [f"{value:.2f}" for value in np.sort(unique_cycles_in_days)]
    print("Unique cycles:", formatted_cycles, "in days")
    # Format the output to show in hours without scientific notation
    formatted_cycles_in_hours = [f"{value:.2f}" for value in np.sort(unique_cycles_in_hours)]
    print("Unique cycles:", formatted_cycles_in_hours, "in hours")

    for cycle in unique_cycles_in_hours:
        df_with_preds[f"hr_{cycle:.2f}h_sin"] = np.sin(2 * np.pi * df_with_preds['hour_of_day_half'] / cycle)
        df_with_preds[f"hr_{cycle:.2f}h_cos"] = np.cos(2 * np.pi * df_with_preds['hour_of_day_half'] / cycle)


    # Month, Quarter, and Seasons
    df_with_preds['year'] = df_with_preds['Datetime'].dt.year
    df_with_preds['month'] = df_with_preds['Datetime'].dt.month
    df_with_preds['quarter'] = df_with_preds['Datetime'].dt.quarter

    # Reset Datetime as the index
    df_with_preds = df_with_preds.set_index('Datetime')

    df_with_preds = df_with_preds.drop(columns=["prev_day_close"])
    df_with_preds = df_with_preds.drop(columns=[col for col in df_with_preds.columns if col.endswith("_diff")])

    cols_not_to_differentiate = ["year", "month", "quarter", "hour_of_day_half", "hour_of_day" ,"day_of_week" ,  # remove the time ones
                                "is_peak_hour", "is_night", "is_weekend",  # remove the boolean ones
                                "open_price", "close_price", "prev_day_close",  # remove the price ones
                                "LoLP_lag1", "System_Price_lag_1", "System_Price_lag_2", "Day_Ahead_Price_lag_48", "NIV_Outturn_lag_1", "NIV_Outturn_lag_2"]  # remove the lag ones
    cols_not_to_differentiate = cols_not_to_differentiate + not_predicted_columns  # We also do not do differentiation on the cols we did not predict

    for col in [cols for cols in df_with_preds.columns if not cols.endswith("sin") and not cols.endswith("cos")]:
        if col not in cols_not_to_differentiate:
            df_with_preds.loc[:,f"{col}_diff"] = df_with_preds[col].diff()
    
    return df_with_preds, Day_Ahead_Price_lag_48, not_predicted_columns

# User Defined Functions for ML

### Auto ARIMA Model

In [25]:
def statsforecast_arima(df, forecast_horizon):
    """
    Fit an AutoARIMA model to the time series data and forecast future values.

    This function uses AutoARIMA from the statsforecast package to automatically select the best ARIMA model.
    It performs both in-sample prediction and forecasts future values beyond the length of the data provided.

    Parameters:
    - df (Series): The input time series data. The index must be a DateTimeIndex.

    Returns:
    - DataFrame: A DataFrame containing the in-sample predictions and forecasted values over an extended range.
    """
    # Ensure the input is a pandas DataFrame with required columns
    df = pd.DataFrame({'unique_id': 1, 'ds': df.index, 'y': df.values})

    # Initialize the StatsForecast object with the AutoARIMA model
    sf = StatsForecast(models=[AutoARIMA()], freq='30min', n_jobs=-1)

    # Forecast future values
    forecast = sf.forecast(df=df, h=forecast_horizon, fitted=True)
    values=sf.forecast_fitted_values()
    values.set_index('ds', inplace=True)
    forecast.set_index('ds', inplace=True)
    result = pd.concat([values, forecast])
    return result["AutoARIMA"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def make_future_predictions_multivariate(model, data, exog_future, n_future_steps, time_step):
    """
    Generates future predictions using a trained LSTM model with future exogenous data.

    Parameters:
    - model: The trained LSTM model.
    - data (array-like): The input time series data (target variable), shape [time_steps, 1].
    - exog_future (array-like): Future exogenous variables, shape [n_future_steps, num_features].
    - n_future_steps (int): The number of future steps to predict.
    - time_step (int): The length of the input sequence required by the LSTM model.

    Returns:
    - future_preds (np.array): Predicted target values for the next `n_future_steps`.
    """
    future_preds = []

    # Initialize input sequence with the last `time_step` rows of `data`
    input_seq = data[-time_step:].reshape(1, time_step, 1)
    
    for i in range(n_future_steps):
        # Predict the next target value
        pred = model.predict(input_seq)[0, 0]  # Predict for the target variable
        future_preds.append(pred)

        # Update the input sequence
        # Combine prediction (target) with future exogenous data for the next time step
        future_exog = exog_future[i].reshape(1, 1, -1)  # Exogenous variables for the next step
        pred_with_exog = np.concatenate((pred.reshape(1, 1, 1), future_exog), axis=2)
        
        # Update input_seq: Remove the oldest step and append the new prediction and exogenous features
        input_seq = np.append(input_seq[:, 1:, :], pred_with_exog, axis=1)
    return np.array(future_preds)
 
def create_dataset_with_exog(data, exog_data, time_step=1):
    """
    Creates a dataset for time series modeling, including exogenous variables.

    Parameters:
    - data (np.array): The main time series data (e.g., the target variable), assumed to be a 2D array.
    - exog_data (np.array): Exogenous variables (features other than the target), assumed to be a 2D array.
    - time_step (int): The number of time steps in each input sequence.

    Returns:
    - X (np.array): Input sequences for the target variable, of shape [samples, time_step].
    - exog_X (np.array): Input sequences for exogenous variables, of shape [samples, time_step, exog_features].
    - y (np.array): Target values for the next time step, of shape [samples].

    Function Details:
    1. **Input Sequences (`X`)**:
       - Extracts sequences of length `time_step` from the target variable (`data`).
       - Each sequence represents the target's past `time_step` values.

    2. **Exogenous Sequences (`exog_X`)**:
       - Extracts sequences of length `time_step` from the exogenous data (`exog_data`).
       - Includes all features in the exogenous dataset.

    3. **Target Values (`y`)**:
       - Sets the target as the value immediately following each input sequence (`data[i + time_step, 0]`).
    """
    X, exog_X, y = [], [], []
    for i in range(len(data) - time_step):
        # Correctly align sequences and target
        X.append(data[i:(i + time_step), 0])  # Input sequence
        exog_X.append(exog_data[i:(i + time_step), :])  # Exogenous sequence
        y.append(data[i + time_step, 0])  # Target is the next step
    return np.array(X), np.array(exog_X), np.array(y)


def inverse_transform_preds(array, target_scaler):
    """
    Reverses the scaling applied to predictions to restore them to the original scale.
    
    Parameters:
    - array (np.array): The scaled predictions.
    - target_scaler: The scaler object used for scaling the target variable.
    Returns:
    - final_preds (np.array): The predictions in the original scale.

    Details:
    - Reshapes the scaled predictions into a 2D array to use the scaler's `inverse_transform` method.
    - Flattens the resulting array back to 1D after the inverse transformation.
    - Removes the first element from the predictions due to a mismatch during alignment.
    """
    final_preds = target_scaler.inverse_transform(array.reshape(-1, 1))
    final_preds = final_preds.flatten()
    final_preds = final_preds[1:]  # Shift the array by removing the first element, as there is an error in matching.
    return final_preds


def LSTM_prepare_data(df_with_preds, time_step, test_size, cols_to_drop, target):
   """
   Prepares data for training and testing an LSTM model with exogenous variables, including scaling and sequence creation.

   Parameters:
   - df_with_preds (pd.DataFrame): The input DataFrame containing the target variable and exogenous variables.
   - time_step (int): The number of time steps in each input sequence.
   - test_size (int): The number of rows to use for the test set.
   - cols_to_drop (list): List of columns to drop from the dataset.

   Returns:
   - target_scaler (StandardScaler): Fitted scaler for the target variable.
   - X_train (np.array): Training input sequences for the target variable.
   - X_test (np.array): Testing input sequences for the target variable.
   - exog_X_train (np.array): Training input sequences for exogenous variables.
   - exog_X_test (np.array): Testing input sequences for exogenous variables.
   - y_train (np.array): Training target sequences.
   - y_test (np.array): Testing target sequences.
   - index (pd.Index): Index of the original DataFrame.

   Function Details:
   1. **Data Cleaning**:
      - Drops specified columns (`cols_to_drop`) from the DataFrame.
      - Removes the first row, which may contain null values due to lag features.

   2. **Train/Test Split**:
      - Splits the dataset into training and testing sets based on `test_size`.

   3. **Scaling**:
      - Normalizes the target variable using `MinMax`.
      - Normalizes exogenous variables separately using another `MinMax`.

   4. **Sequence Creation**:
      - Creates input sequences of length `time_step` for both the target variable and exogenous variables.
      - Aligns each sequence with the corresponding target value.

   5. **Dataset Splitting**:
      - Splits the sequences back into training and testing sets after sequence creation.

   6. **Reshaping for LSTM**:
      - Reshapes the target input sequences to be compatible with LSTM input format: `[samples, time steps, features]`.
   """
   # Assume df_with_preds is your DataFrame with target as target and other columns as exogenous variables.
   df = df_with_preds.copy()

   if cols_to_drop:
      cols_to_drop = [col for col in cols_to_drop if col != "NIV_Outturn"]
      cols_to_drop = [col for col in cols_to_drop if col != "System_Price"]
      df = df.drop(columns=cols_to_drop, errors="ignore")

   df = df.iloc[2:]  # Drop the first 2 rows (too many nulls)

   # Split the dataset into training and test sets BEFORE scaling
   train_data = df.iloc[:-test_size]
   test_data = df.iloc[-test_size:]

   # Separate target and features for train and test
   y_train_raw = train_data[[target]].values
   y_test_raw = test_data[[target]].values

   X_train_raw = train_data.drop(columns=[target]).values
   X_test_raw = test_data.drop(columns=[target]).values

   # Normalize the target (target) and the exogenous features using StandardScaler
   target_scaler = MinMaxScaler()
   # Fit and transform the target variable (y_train_raw)
   y_train_scaled = target_scaler.fit_transform(y_train_raw)
   # Transform the test target variable (y_test_raw)
   y_test_scaled = target_scaler.transform(y_test_raw)


   exog_scaler = MinMaxScaler()
   # Fit and transform the exogenous variables (X_train_raw)
   X_train_scaled = exog_scaler.fit_transform(X_train_raw)
   # Transform the test exogenous variables (X_test_raw)
   X_test_scaled = exog_scaler.transform(X_test_raw)

   # Concatenate train and test scaled data
   X_full_scaled = np.vstack((X_train_scaled, X_test_scaled))
   y_full_scaled = np.vstack((y_train_scaled, y_test_scaled))

   # Generate the dataset
   X_full, exog_X_full, y_full = create_dataset_with_exog(y_full_scaled, X_full_scaled, time_step)

   # Determine split index (total samples in the train set after sequence creation)
   split_index = len(X_train_scaled) - time_step

   # Split the dataset back into train and test sets
   X_train, X_test = X_full[:split_index], X_full[split_index:]
   exog_X_train, exog_X_test = exog_X_full[:split_index], exog_X_full[split_index:]
   y_train, y_test = y_full[:split_index], y_full[split_index:]


   # Reshape LSTM input to be [samples, time steps, features]
   X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
   X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
   
   X_train_combined = np.concatenate([X_train, exog_X_train], axis=2)
   X_test_combined = np.concatenate([X_test, exog_X_test], axis=2)
   
   index = df.index


   # Step 3: Build and Train the LSTM Model
   # Ensure the model runs on GPU
   lstm_model = Sequential()
   # Add the Input layer
   lstm_model.add(Input(shape=(time_step, 1 + len(X_train.columns))))
   # Add LSTM layer
   lstm_model.add(LSTM(96, return_sequences=True))
   lstm_model.add(LSTM(96, return_sequences=False))
   # Add Dense layer
   lstm_model.add(Dense(18, activation="relu"))
   lstm_model.add(Dense(1, activation="linear"))

   # Compile the LSTM model
   lstm_model.compile(optimizer='adam', loss='mean_squared_error')

   # Train the LSTM model
   lstm_model.fit(X_train_combined, y_train, epochs=3, batch_size=96, verbose=2)

   # Step 4: Make Predictions with LSTM and Calculate Residuals
   # Predict on training and test sets
   y_train_pred = lstm_model.predict(X_train_combined)   

   # Make future predictions
   n_future_steps = len(X_test_scaled) + 1  # Predict for all test steps
   future_preds = make_future_predictions_multivariate(
      model=lstm_model,
      data=y_train_scaled,
      exog_future=X_test_scaled,
      n_future_steps=n_future_steps,
      time_step=time_step
   )

   # Inverse transform the predictions to original scale
   final_train_preds = inverse_transform_preds(y_train_pred, target_scaler)
   final_test_preds = inverse_transform_preds(future_preds, target_scaler)
   y_train_original = target_scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
   
   # Calculate residuals
   train_residuals = y_train_original[1:] - final_train_preds

   # Step 5: Evaluate the Model
   # Calculate RMSE
   rmse = np.sqrt(mean_squared_error(y_train_original[1:], final_train_preds))
   print(f"Test RMSE: {rmse:.2f}")

   # Plot the results
   plt.figure(figsize=(20, 10))
   plt.plot(index[:-49][time_step+17470:], y_train_original[17469:-1], label=f'Actual {target}')
   plt.plot(index[:-49][time_step+17470:], final_train_preds[17469:], label='LSTM', linestyle='dashed')
   plt.title(f'{target} Forecasting (LSTM)')
   plt.xlabel('Date')
   plt.ylabel(target)
   plt.legend()
   plt.show()
   
   return final_train_preds, final_test_preds, train_residuals


In [ ]:
df_with_preds, Day_Ahead_Price_lag_48, not_predicted_columns = feature_engineering(merged_df_2024)  # Fill in t+1 to t+48 and add lagged values for the cols we can't predict
df_with_preds = reduce_mem_usage(df_with_preds, verbose=True)  # reduce memory usage so there is more memory for computing
temp_df = df_with_preds.copy()
Day_Ahead_Price_lag_48.index = pd.to_datetime(Day_Ahead_Price_lag_48.index )
dap48 = df_with_preds.loc["2024-09-30"]["Day_Ahead_Price"]

Working on Intraday_Price


MemoryError: Unable to allocate 52.2 GiB for an array with shape (118320, 118320) and data type int32

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Conv1D,
    MaxPooling1D,
    LSTM,
    Dense,
    Input,
    Dropout,
    BatchNormalization,
    MultiHeadAttention,
    GlobalAveragePooling1D,
    LayerNormalization,
    Layer,
    Bidirectional
)
import tensorflow.keras.backend as K

@register_keras_serializable()
class TransformerEncoder(Layer):
    def __init__(self, num_heads, key_dim, dense_dim, dropout_rate=0.1, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dense_dim = dense_dim
        self.dropout_rate = dropout_rate

        self.input_proj = Dense(num_heads * key_dim)  # Project inputs to match MHA dimension
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.dense_proj = tf.keras.Sequential([
            Dense(dense_dim, activation="relu"),
            Dense(num_heads * key_dim)  # matches MHA output dimension
        ])
        self.dropout = Dropout(dropout_rate)
        self.layernorm1 = LayerNormalization()
        self.layernorm2 = LayerNormalization()

    def call(self, inputs, training=None):
        # Project inputs to (None, seq_len, num_heads*key_dim)
        inputs_proj = self.input_proj(inputs)
        # MultiHeadAttention output: (None, seq_len, num_heads*key_dim)
        attn_output = self.attention(inputs_proj, inputs_proj)
        # Residual connection: both inputs_proj and attn_output are (None, seq_len, num_heads*key_dim)
        attn_output = self.layernorm1(inputs_proj + attn_output)
        proj_output = self.dense_proj(attn_output)
        proj_output = self.dropout(proj_output, training=training)
        # Another residual: attn_output and proj_output now have the same shape
        return self.layernorm2(attn_output + proj_output)

    def get_config(self):
        config = super(TransformerEncoder, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
            "dense_dim": self.dense_dim,
            "dropout_rate": self.dropout_rate,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


@register_keras_serializable()
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name="W",
            shape=(input_shape[-1], input_shape[-1]),
            initializer=tf.keras.initializers.GlorotUniform(),
            trainable=True,
        )
        self.b = self.add_weight(
            name="b",
            shape=(input_shape[-1],),
            initializer=tf.keras.initializers.Zeros(),
            trainable=True,
        )
        self.u = self.add_weight(
            name="u",
            shape=(input_shape[-1],),
            initializer=tf.keras.initializers.GlorotUniform(),
            trainable=True,
        )
        super(Attention, self).build(input_shape)

    def call(self, inputs):
        # Compute the score
        score = tf.tanh(tf.matmul(inputs, self.W) + self.b)
        # Compute attention weights
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.u, axes=1), axis=1)
        # Compute the context vector
        context_vector = tf.reduce_sum(attention_weights[:, :, tf.newaxis] * inputs, axis=1)
        return context_vector

    def compute_output_shape(self, input_shape):
        # Output shape is (batch_size, input_dim)
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        base_config = super(Attention, self).get_config()
        return base_config


@register_keras_serializable()
class PositionalEncoding(Layer):
    def __init__(self, sequence_length, model_dim, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.sequence_length = sequence_length
        self.model_dim = model_dim if model_dim % 2 == 0 else model_dim + 1  # Ensure even dimension

    def build(self, input_shape):
        position = np.arange(self.sequence_length)[:, np.newaxis]
        div_term = np.exp(np.arange(0, self.model_dim, 2) * -(np.log(10000.0) / self.model_dim))
        pos_encoding = np.zeros((self.sequence_length, self.model_dim))
        pos_encoding[:, 0::2] = np.sin(position * div_term)
        pos_encoding[:, 1::2] = np.cos(position * div_term)

        self.pos_encoding = tf.constant(pos_encoding, dtype=tf.float32)
        super(PositionalEncoding, self).build(input_shape)

    def call(self, inputs):
        # Inputs: (batch_size, sequence_length, model_dim)
        # Add positional encoding to input
        seq_len = tf.shape(inputs)[1]
        feature_dim = tf.shape(inputs)[2]
        # If feature_dim < self.model_dim, slice pos_encoding
        pos_enc = self.pos_encoding[:, :feature_dim]
        pos_enc = tf.expand_dims(pos_enc, 0)  # (1, sequence_length, feature_dim)
        pos_enc = pos_enc[:, :seq_len, :]  # in case input seq length < self.sequence_length
        return inputs + pos_enc

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super(PositionalEncoding, self).get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "model_dim": self.model_dim,
        })
        return config


In [ ]:
from sklearn.preprocessing import StandardScaler

# User-defined parameters
target = "System_Price"
time_step = 48
val_size = 48 * 30
test_size = 49
cols_to_drop = [
    'Hydro_Run-of-River_and_Poundage_missing',
    'Total_Load_missing',
    'Fossil_Oil',
    'Day_Ahead_Price',
    'BM_Offer_Acceptances',
    'Wind_Offshore_missing',
    'Fossil_Oil_missing',
    'Day_Ahead_Price_missing',
    'Fossil_Hard_Coal_missing',
    'System_Price_missing',
    'Fossil_Gas_missing',
    'NIV_Outturn_missing',
    'BM_Bid_Acceptances_missing',
    'Solar_missing',
    'Loss_of_Load_Prob',
    'Intraday_Price_missing',
    'Hydro_Pumped_Storage_missing',
    'BSAD_Turn_Down_missing',
    'BM_Offer_Acceptances_missing',
    'BSAD_Turn_Up_missing',
    'Loss_of_Load_Prob_missing',
    'BSAD_Total_missing',
    'EPEX_Intraday_Volume_missing',
    'BM_Bid_Acceptances',
    'Wind_Onshore_missing',
    'value',
    'Demand_Outturn_missing',
    'Biomass_missing',
    'Nuclear_missing',
]

not_predicted_columns = ['Hydro_Pumped_Storage',
 'Wind_Offshore',
 'BSAD_Turn_Up',
 'BSAD_Total',
 'Biomass',
 'Fossil_Gas',
 'Wind_Onshore',
 'BM_Offer_Acceptances',
 'Loss_of_Load_Prob',
 'Nuclear',
 'Solar',
 'Day_Ahead_Price',
 'Fossil_Hard_Coal',
 'Hydro_Run-of-River_and_Poundage',
 'value',
 'BM_Bid_Acceptances',
 'Fossil_Oil',
 'BSAD_Turn_Down']

cols_to_drop = cols_to_drop + not_predicted_columns

# Load DataFrame
df = temp_df # pd.read_csv("trying333.csv")
df.drop(columns=["Unnamed: 0", "\tUnnamed: 0"], errors="ignore", inplace=True)
# df.set_index("Datetime", inplace=True)
df.index = pd.to_datetime(df.index)
# df.loc["2024-10-01", "System_Price"] = df.loc["2024-09-30"]["Day_Ahead_Price"].values  # placeholder set with DAP lag 48

# Drop unwanted columns
if cols_to_drop:
    cols_to_drop = [col for col in cols_to_drop if col not in ["NIV_Outturn", "System_Price"]]
    df = df.drop(columns=cols_to_drop, errors="ignore")

df = df.iloc[2:]  # Drop first 2 rows
df = df.drop(columns=["hr_17664.96h_cos", "hr_17664.96h_sin", "open_price", "close_price", "Wind_Solar", "Wind_Solar_diff"] + [col for col in df.columns if col.endswith("diff")])
# Define the targets
targets = ["System_Price", "NIV_Outturn"]

# Split data indices
split_index_val = len(df) - test_size - val_size
split_index_test = len(df) - test_size

if split_index_val <= time_step or split_index_val <= 0:
    raise ValueError("Not enough data for training and validation sets with the given time_step and val_size.")

# Split the data
train_data = df.iloc[:split_index_val]
val_data = df.iloc[split_index_val:split_index_test]
test_data = df.iloc[split_index_test:]

# Separate targets and features
y_train_raw = train_data[targets].values
y_val_raw = val_data[targets].values

X_train_raw = train_data.drop(columns=targets).values
X_val_raw = val_data.drop(columns=targets).values
X_test_raw = test_data.drop(columns=targets).values

# Define separate scalers for targets and features
scaler_y = StandardScaler()
scaler_X = StandardScaler()

y_train_scaled = scaler_y.fit_transform(y_train_raw)
y_val_scaled = scaler_y.transform(y_val_raw)

X_train_scaled = scaler_X.fit_transform(X_train_raw)
X_val_scaled = scaler_X.transform(X_val_raw)
X_test_scaled = scaler_X.transform(X_test_raw)

# Update dataset creation function to handle multivariate targets
def create_dataset_with_exog(Y, X_exog, time_step):
    X, exog_X, Y_multi = [], [], []
    for i in range(len(Y) - time_step):
        X.append(Y[i:(i + time_step), :])  # Multivariate input sequence for targets
        exog_X.append(X_exog[i:(i + time_step), :])  # Exogenous features
        Y_multi.append(Y[i + time_step, :])  # Multivariate target (next step)
    return np.array(X), np.array(exog_X), np.array(Y_multi)


# Create single-step dataset
X_train, exog_X_train, y_train = create_dataset_with_exog(y_train_scaled, X_train_scaled, time_step)
X_val, exog_X_val, y_val = create_dataset_with_exog(y_val_scaled, X_val_scaled, time_step)

# Combine target and exogenous features
X_train_combined = np.concatenate([X_train, exog_X_train], axis=2)  # Combine target sequence and exogenous features
X_val_combined = np.concatenate([X_val, exog_X_val], axis=2)

In [ ]:
# Build the model (one-step output)
input_layer = Input(shape=(time_step, X_train_combined.shape[2]))
input_with_positional = PositionalEncoding(sequence_length=time_step, model_dim=X_train_combined.shape[2])(input_layer)

cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(input_with_positional)
cnn_layer = MaxPooling1D(pool_size=4)(cnn_layer)
cnn_layer = BatchNormalization()(cnn_layer)

lstm_layer = Bidirectional(LSTM(64, return_sequences=True, activation='tanh', recurrent_activation='sigmoid',
                                dropout=0.1, kernel_regularizer=l2(0.001)))(cnn_layer)
lstm_layer = LayerNormalization()(lstm_layer)

transformer_out = TransformerEncoder(num_heads=4, key_dim=128, dense_dim=256)(lstm_layer)
attention_out = MultiHeadAttention(num_heads=4, key_dim=64)(transformer_out, transformer_out)
pooled_output = GlobalAveragePooling1D()(attention_out)

dense_out = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(pooled_output)
dense_out = Dropout(0.1)(dense_out)

final_output = Dense(2, activation='linear')(dense_out)

lstm_model = Model(inputs=input_layer, outputs=final_output)
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(optimizer=optimizer, loss='mean_squared_error')

checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Train the model on single-step prediction
lstm_model.fit(
    X_train_combined, y_train,
    validation_data=(X_val_combined, y_val),
    epochs=20,
    batch_size=96,
    verbose=2,
    callbacks=[checkpoint_callback, reduce_lr, early_stopping]
)

# Load the best model
lstm_model_best_val = load_model('best_model.keras', custom_objects={'Attention': Attention,
        "TransformerEncoder": TransformerEncoder,
        "PositionalEncoding": PositionalEncoding,
    })

###############################################################################
# Iterative Forecasting Code (Real Implementation)
###############################################################################
# Let's assume we want to predict the next 48 steps into the future.
n_future_steps = 48

# Start from the last known sequence in the validation set as our base.
# This represents the last window we have real data for.
last_known_sequence = X_val_combined[-1:].copy()  # Shape: (1, time_step, features)

predictions = []
for i in range(n_future_steps):
    # Predict the next step (scaled)
    next_pred_scaled = lstm_model_best_val.predict(last_known_sequence, verbose=0)  # Shape: (1,1)

    # Inverse transform to original scale
    next_pred = scaler_y.inverse_transform(next_pred_scaled.reshape(-1,2))
    predictions.append(next_pred[0])

    # Get the exogenous features for the next step from test (or future) data
    # Make sure you have at least `n_future_steps` rows in X_test_exog_scaled
    exog_next_step = X_test_scaled[i].reshape(1,1,-1) 

    # Re-scale the predicted value to append back into the model's input domain
    next_pred_scaled_value = scaler_y.transform(next_pred)  # Shape: (1, 2)

    # Construct the next input step (predicted target + future exogenous)
    next_input = np.concatenate([next_pred_scaled_value.reshape(1,1,-1), exog_next_step], axis=2)

    # Update the sequence: remove the oldest time step and add the new predicted time step
    last_known_sequence = np.append(last_known_sequence[:,1:,:], next_input, axis=1)

# 'predictions' now contains the iteratively predicted values for the next 48 steps.
###############################################################################

# Evaluate predictions as needed (e.g., plotting).
print("Future predictions:", np.array(predictions))

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.83870, saving model to best_model.keras
168/168 - 45s - 265ms/step - loss: 1.0399 - val_loss: 0.8387 - learning_rate: 0.0010
Epoch 2/20

Epoch 2: val_loss improved from 0.83870 to 0.72778, saving model to best_model.keras
168/168 - 15s - 90ms/step - loss: 0.8632 - val_loss: 0.7278 - learning_rate: 0.0010
Epoch 3/20

Epoch 3: val_loss did not improve from 0.72778
168/168 - 15s - 87ms/step - loss: 0.7835 - val_loss: 0.7524 - learning_rate: 0.0010
Epoch 4/20

Epoch 4: val_loss improved from 0.72778 to 0.70200, saving model to best_model.keras
168/168 - 15s - 89ms/step - loss: 0.7470 - val_loss: 0.7020 - learning_rate: 0.0010
Epoch 5/20

Epoch 5: val_loss improved from 0.70200 to 0.69913, saving model to best_model.keras
168/168 - 15s - 91ms/step - loss: 0.7066 - val_loss: 0.6991 - learning_rate: 0.0010
Epoch 6/20

Epoch 6: val_loss improved from 0.69913 to 0.65300, saving model to best_model.keras
168/168 - 15s - 91ms/step - loss: 0.678

c:\Users\ruben\anaconda3\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Future predictions: [[  70.030815    71.44727  ]
 [  66.09799    143.57872  ]
 [  69.29823     56.79961  ]
 [  60.910717   137.15723  ]
 [  57.599117   207.4843   ]
 [  58.328514   168.89569  ]
 [  51.931633   246.61555  ]
 [  55.972588   193.10384  ]
 [  63.90398    133.88945  ]
 [  65.00756    127.71368  ]
 [  65.97365    141.7727   ]
 [  66.11592     85.726135 ]
 [  69.90997     94.63802  ]
 [  74.66951     57.8639   ]
 [  80.876274   -29.288733 ]
 [  79.55052     14.488961 ]
 [  78.20144     31.634426 ]
 [  76.413506    58.811577 ]
 [  71.917244   103.4092   ]
 [  70.07827    106.55579  ]
 [  68.94353    118.28267  ]
 [  67.07964    134.35764  ]
 [  66.2452     164.91531  ]
 [  66.3921     152.8478   ]
 [  68.36144    119.70413  ]
 [  68.43518    119.46371  ]
 [  70.254845    95.75823  ]
 [  70.754486    88.88231  ]
 [  72.65156     67.77291  ]
 [  74.10291     41.341526 ]
 [  76.94395      5.6208124]
 [  78.12613    -11.221683 ]
 [  80.146774   -24.71676  ]
 [  84.44823    -73.456

In [ ]:
temp = np.array(predictions)
predictions_sep = pd.DataFrame({"System_Price": temp[:, 0], "NIV_Outturn": temp[:, 1]})

In [ ]:
SP_preds =  np.array(predictions_sep["System_Price"]) * 0.74 + np.array(dap48)*0.46

NIV_preds = -220 - 3.5*np.array(predictions_sep["System_Price"]).reshape(-1,1)

Root Mean Squared Error System Price (RMSE): 15.080361514332411
Root Mean Squared Error NIV Outturn (RMSE): 414.514687344532
